In [27]:
#libraries
import pandas as pd
import numpy as np
import datetime as dt
pd.set_option("display.max_rows", None, "mode.chained_assignment", None)
import numpy as np
import matplotlib.pyplot as plt
import json
from json import loads, dumps
import datetime as dt
import plotly
import plotly.express as px

In [28]:
# read csv
person = pd.read_csv('../synpuff/person.csv')
condition_occurrence = pd.read_csv('../synpuff/condition_occurrence.csv')
drug_exposure = pd.read_csv('../synpuff/drug_exposure.csv')
concept = pd.read_csv('../synpuff/concept.csv')

In [29]:
#concept lookup in concept df
concept_lookup = {c.concept_id: c.concept_name for c in concept.itertuples()}

In [30]:
#function to make labels
def make_labels(df):
    for c in df.columns:
        if 'concept_id' in c:
            df[c.replace('_id', '_label')] = df[c].map(concept_lookup)
        if 'concept_id' in c or 'source' in c or len(df[df[c].notna()])==0:
            df = df.drop(c, axis=1)
    return df

In [31]:
#
person_labelled = make_labels(person)
condition_occurrence_labelled = make_labels(condition_occurrence)
drug_exposure_labelled = make_labels(drug_exposure)
measurement_labelled = pd.read_csv('../synpuff/measurement.csv')
procedure_occurrence_labelled = pd.read_csv('../synpuff/procedure.csv')

In [32]:
person_code = 1705545

In [33]:
meas_indiv = measurement_labelled.loc[measurement_labelled['person_id']==person_code]

In [34]:
### measurements
meas_weight = meas_indiv.loc[meas_indiv['measurement_concept_label'].isin(['Body weight','Weight change'])]
#Merge weight measurements with weight differences from last measurement
meas_weight['value_as_number']=meas_weight['value_as_number'].apply(str)
meas_grouped = meas_weight.groupby(['measurement_date'])['value_as_number'].transform(lambda x: ' kg,<br>Weight change: '.join(x))
meas_weight['value_as_number'] = meas_grouped
meas_weight['content'] = 'Body weight: ' + meas_weight['value_as_number'] + ' kg'
#Rename columns
meas_weight.rename(columns={'measurement_date':'start'}, inplace=True)
meas_weight['group'] = 'measurement'
meas_weight['className'] = 'measurements'
meas_weight.drop_duplicates(subset=['content', 'start'],keep='last',inplace=True)

In [35]:
meas_indiv

,measurement_id,person_id,measurement_date,measurement_datetime,measurement_time,value_as_number,measurement_concept_label,unit_concept_label,measurement_type_concept_label
0,f58d40ef-6d5e-bd81-7ebb-5eedd2c6ed97,1705545,2008-01-27,2008-01-27 16:02:00,16:02:00,162.4,Body height,cm,Vital signs
1,f58d40ef-6d5e-bd81-7ebb-5eedd2c6ed97,1705545,2008-01-27,2008-01-27 16:02:00,16:02:00,76.3,Body mass index (BMI) [Percentile] Per age and...,%,Vital signs
2,f58d40ef-6d5e-bd81-7ebb-5eedd2c6ed97,1705545,2008-01-27,2008-01-27 16:02:00,16:02:00,21.2,Body mass index (BMI) [Ratio],kg-m2,Vital signs
3,f58d40ef-6d5e-bd81-7ebb-5eedd2c6ed97,1705545,2008-01-27,2008-01-27 16:02:00,16:02:00,55.8,Body weight,kg,Vital signs
4,f1a24503-271b-a86e-2b33-47bc192d20c0,1705545,2008-04-01,2008-04-01 16:02:00,16:02:00,165.5,Body height,cm,Vital signs
5,f1a24503-271b-a86e-2b33-47bc192d20c0,1705545,2008-04-01,2008-04-01 16:02:00,16:02:00,73.3,Body mass index (BMI) [Percentile] Per age and...,%,Vital signs
6,f1a24503-271b-a86e-2b33-47bc192d20c0,1705545,2008-04-01,2008-04-01 16:02:00,16:02:00,21.5,Body mass index (BMI) [Ratio],kg-m2,Vital signs
7,f1a24503-271b-a86e-2b33-47bc192d20c0,1705545,2008-04-01,2008-04-01 16:02:00,16:02:00,59.0,Body weight,kg,Vital signs
8,f1a24503-271b-a86e-2b33-47bc192d20c0,1705545,2008-04-01,2008-04-01 16:02:00,16:02:00,5.7,Weight change,percent,Vital signs
9,2c88945b-4a7d-9493-74ee-7c5e464430ac,1705545,2008-06-09,2008-06-09 16:02:00,16:02:00,166.8,Body height,cm,Vital signs


In [36]:
meas_graphs = meas_indiv.loc[meas_indiv['measurement_concept_label'].isin(['Body mass index (BMI) [Ratio]', 'Body height', 'Body weight', 'Weight change'])]
meas_graphs

,measurement_id,person_id,measurement_date,measurement_datetime,measurement_time,value_as_number,measurement_concept_label,unit_concept_label,measurement_type_concept_label
0,f58d40ef-6d5e-bd81-7ebb-5eedd2c6ed97,1705545,2008-01-27,2008-01-27 16:02:00,16:02:00,162.4,Body height,cm,Vital signs
2,f58d40ef-6d5e-bd81-7ebb-5eedd2c6ed97,1705545,2008-01-27,2008-01-27 16:02:00,16:02:00,21.2,Body mass index (BMI) [Ratio],kg-m2,Vital signs
3,f58d40ef-6d5e-bd81-7ebb-5eedd2c6ed97,1705545,2008-01-27,2008-01-27 16:02:00,16:02:00,55.8,Body weight,kg,Vital signs
4,f1a24503-271b-a86e-2b33-47bc192d20c0,1705545,2008-04-01,2008-04-01 16:02:00,16:02:00,165.5,Body height,cm,Vital signs
6,f1a24503-271b-a86e-2b33-47bc192d20c0,1705545,2008-04-01,2008-04-01 16:02:00,16:02:00,21.5,Body mass index (BMI) [Ratio],kg-m2,Vital signs
7,f1a24503-271b-a86e-2b33-47bc192d20c0,1705545,2008-04-01,2008-04-01 16:02:00,16:02:00,59.0,Body weight,kg,Vital signs
8,f1a24503-271b-a86e-2b33-47bc192d20c0,1705545,2008-04-01,2008-04-01 16:02:00,16:02:00,5.7,Weight change,percent,Vital signs
9,2c88945b-4a7d-9493-74ee-7c5e464430ac,1705545,2008-06-09,2008-06-09 16:02:00,16:02:00,166.8,Body height,cm,Vital signs
11,2c88945b-4a7d-9493-74ee-7c5e464430ac,1705545,2008-06-09,2008-06-09 16:02:00,16:02:00,21.7,Body mass index (BMI) [Ratio],kg-m2,Vital signs
12,2c88945b-4a7d-9493-74ee-7c5e464430ac,1705545,2008-06-09,2008-06-09 16:02:00,16:02:00,60.5,Body weight,kg,Vital signs


In [37]:
meas_graphs['measurement_concept_label']=meas_graphs['measurement_concept_label'].replace({
    'Body weight':'Body weight (kg)', 
    'Weight change':'Weight change (%)',
    'Body height': 'Body height (cm)',
    'Body mass index (BMI) [Ratio]': 'BMI (kg/m\u00B2)'})

In [39]:
weight_fig = px.scatter(meas_graphs, x="measurement_date", y="value_as_number", 
facet_col="measurement_concept_label", facet_col_wrap=2,
facet_row_spacing=0.1, facet_col_spacing=0.05)
weight_fig.update_yaxes(matches=None, showticklabels=True, title="")
weight_fig.update_xaxes(title="")
weight_fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
weight_fig.update_layout(margin=dict(l=0, r=10, t=20, b=10))
weight_fig.show()
weight_fig.write_html("weight-plot.html", full_html=False, include_plotlyjs=False)